In [30]:
# these pakages are need to run ingoogle colab


#!pip install pytesseract
#!sudo apt-get install tesseract-ocr
#!pip install pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [61]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
# Load the image
image = cv2.imread('bill.jpg')



# Determine the dimensions of the image
height, width, channels = image.shape
print(image.shape)

# Divide the image into segmentation
top_half = image[:380, :, :]
middle_half = image[320:480,:,:]
middle_two = image[320:980,:750,:]
bottom_half = image[820:, :, :]

# segmentation of image for preprocessing
cv2.imwrite('top_half.jpg', top_half)

cv2.imwrite('middle_half.jpg', middle_half)

cv2.imwrite('middle_half2.jpg', middle_two)

cv2.imwrite('bottom_half.jpg', bottom_half)


(1024, 768, 3)


True

In [62]:
bill_text1 = pytesseract.image_to_string(top_half)
bill_text2 = pytesseract.image_to_string(middle_half)
bill_text3 = pytesseract.image_to_string(middle_two)
bill_text4 = pytesseract.image_to_string(bottom_half)
print(bill_text1)

MARCO POLO ORTIGAS MANILA
Owned by
FRONTIER ORTIGAS

HOTEL and RESORT CORP.
Meralco Ave. & Sapphire Rd.

Ortigas Ctr., Pasig City

1600 Philippines

VAT REG TIN: 008-611-772-000

 

LUNG HIN
Table: Ri NPE »7WNHEO—HH

  



In [63]:
import pytesseract
from PIL import Image

#generate the texts from the images
bill_text1 = pytesseract.image_to_string(image)
bill_text2 = pytesseract.image_to_string(middle_half)
bill_text3 = pytesseract.image_to_string(middle_two)
bill_text4 = pytesseract.image_to_string(bottom_half)


import re
import json

# The text from the image
"""
MARCO POLO ORTIGAS MANILA
Owned by
HOTEL and RESORT CORP.
Meralco Ave. & Sapphire Rd.
Ortigas Ctr., Pasig City
1600 Philippines
VAT REG TIN: 008-611-772-000
"""
bill_text = bill_text1
# Define regular expressions to extract relevant information
shop_name_pattern = r"MARCO POLO ORTIGAS MANILA"
owner_name_pattern = r"Owned by\n([^\n]+)"
address_pattern = r"Owned by([\s\S]+?)VAT REG TIN"
reg_no_pattern = r"VAT REG TIN: (\d{3}-\d{3}-\d{3}-\d{3})"

# Initialize a dictionary to store extracted information
data = {
    "Shop Name": None,
    "Owner Name": None,
    "Address": {
        "Organization": None,
        "Street": None,
        "City": None,
        "Postal Code": None,
        "Country": None
    },
    "Registration Number": None
}

# Extract information using regular expressions
shop_name_match = re.search(shop_name_pattern, bill_text)
if shop_name_match:
    data["Shop Name"] = shop_name_match.group(0)

owner_name_match = re.search(owner_name_pattern, bill_text)
if owner_name_match:
    data["Owner Name"] = owner_name_match.group(1).strip()

address_match = re.search(address_pattern, bill_text)
if address_match:
    address_text = address_match.group(1).strip()
    address_parts = address_text.split('\n')

    if len(address_parts) >= 4:
        data["Address"]["Organization"] = address_parts[0].strip()
        data["Address"]["Street"] = address_parts[1].strip()
        data["Address"]["City"] = address_parts[2].strip()

        postal_code_country = address_parts[-1].split(' ')
        if len(postal_code_country) == 2:
            data["Address"]["Postal Code"] = postal_code_country[0].strip()
            data["Address"]["Country"] = postal_code_country[1].strip()

reg_no_match = re.search(reg_no_pattern, bill_text)
if reg_no_match:
    data["Registration Number"] = reg_no_match.group(1)

# Convert the dictionary to JSON format
output_json1 = json.dumps(data, indent=4)



import re
import json

# The text from the image
bill_text = bill_text2
"""
LUNG HIN
Table: Bi OR# :333099-00
Date +: 08 Feb,i5 Time +: 12:49
Staff : ALEMIR Cover : 3

Kio o0©0 OJ QSMINE 179.57
"""

# Define regular expressions to extract date, time, staff, cover, and table details
#date_pattern = r"Date +: (\d{2} \w+,\w+\d{2})"
#time_pattern = r"Time +: (\d{2}:\d{2})"
date_time_pattern = r"Date \+: ([\d\s\w,:]+) Time \+: ([\d:]+)"

staff_pattern = r"Staff : (\w+) Cover : (\d+)"
table_pattern = r"Table: (\w+) OR# :(\d+)"

# Initialize variables to store extracted details
date = None
time = None
staff_name = None
cover = None
table_name = None
table_number = None

attributes = {
    "Date": None,
    "Time": None,

}

# Extract date, time, staff, cover, and table details using regular expressions

date_time_match = re.search(date_time_pattern, bill_text)
if date_time_match:
    attributes["Date"] = date_time_match.group(1)
    attributes["Time"] = date_time_match.group(2)



staff_match = re.search(staff_pattern, bill_text)
if staff_match:
    staff_name = staff_match.group(1)
    cover = int(staff_match.group(2))

table_match = re.search(table_pattern, bill_text)
if table_match:
    table_name = table_match.group(1)
    table_number = table_match.group(2)

# Store the details in a dictionary
details = {
    "Table": {
        "Name": table_name,
        "Number": table_number
    },

    "Staff": {
        "Name": staff_name,
        "Cover": cover
    }
}

# Convert the dictionary to JSON format
output_json2 = json.dumps(details, indent=4)

output_json3 = json.dumps(attributes, indent=4)



import re
import json

# The text from the image
bill_text = """
VaTable 1770.93
10% S.C. 260.72
12% VAT 212.51
LTax Fd/By 49.59

Total: 2293.75
"""

# Define regular expressions to extract relevant information
vatable_pattern = r"VaTable (\d+\.\d+)"
service_charge_pattern = r"10% S.C. (\d+\.\d+)"
vat_pattern = r"12% VAT (\d+\.\d+)"
ltax_pattern = r"LTax Fd/By (\d+\.\d+)"
total_pattern = r"Total: (\d+\.\d+)"

# Initialize a dictionary to store extracted information
data = {
    "VATable": None,
    "Service Charge": None,
    "VAT": None,
    "LTax": None,
    "Total": None
}

# Extract information using regular expressions
vatable_match = re.search(vatable_pattern, bill_text)
if vatable_match:
    data["VATable"] = float(vatable_match.group(1))

service_charge_match = re.search(service_charge_pattern, bill_text)
if service_charge_match:
    data["Service Charge"] = float(service_charge_match.group(1))

vat_match = re.search(vat_pattern, bill_text)
if vat_match:
    data["VAT"] = float(vat_match.group(1))

ltax_match = re.search(ltax_pattern, bill_text)
if ltax_match:
    data["LTax"] = float(ltax_match.group(1))

total_match = re.search(total_pattern, bill_text)
if total_match:
    data["Total"] = float(total_match.group(1))

# Convert the dictionary to JSON format
output_json5 = json.dumps(data, indent=4)



import re
import json

# The text from the image
bill_text = """
LUNG HIN
Table: Bi OR# :333099-00
Date +: 08 Feb,iS Time +: 12:49
Staff +: ALEMIR Cover : 3

¥1 0 JASMINE 178.57
41 ST.CHX FEET x0 160.71
¥100 ST.PORK SIOMAT 196.43
x10 ST.G RICE ABALONE 196.43

X10 ST.SPINACH DUMPLING © 196.43
a2 ST.SCALLOP DUMPLING 428.58
1 BAKED BRQ PORK BUN 196.43
x1 PANFRIED ONION CAKE 194.43
¥1 FISH SCALLOP SP Px 339.29
41) MAR BEEF GARLIC 517.86
MPM DP 15% Bey petra

MPM Dining Privilege 809.44
a

Sub-Tote 1770.93

VATable 1770.93
10% S.C. 260.72
12% VAT 212.51
LTax 49,59

Total: 2299.75
"""

# Define regular expressions to extract relevant information

item_pattern = r"(\d+) (.+?) (\d+\.\d+)"
subtotal_pattern = r"Sub-Tote (\d+\.\d+)"
mpm_dining_pattern = r"MPM Dining Privilege ([\d\.\d+]+)"

# Initialize variables to store extracted attributes
attributes = {

    "Items": [],
    "Subtotal": None,
    "MPM Dining Privilege": None
}

# Extract and store attributes

item_matches = re.findall(item_pattern, bill_text)
for item_match in item_matches:
    quantity, item_name, price = item_match
    attributes["Items"].append({
        #"Quantity": int(quantity),
        "Item Name": item_name,
        "Price": float(price)
    })

subtotal_match = re.search(subtotal_pattern, bill_text)
if subtotal_match:
    attributes["Subtotal"] = float(subtotal_match.group(1))

mpm_dining_match = re.search(mpm_dining_pattern, bill_text)
if mpm_dining_match:
    attributes["MPM Dining Privilege"] = -float(mpm_dining_match.group(1).replace(',', ''))

# Convert the attributes to JSON
output_json4 = json.dumps(attributes, indent=4)

# Print the JSON output
print(output_json1)
# Print the JSON output
print(output_json2)
# Print the JSON output
print(output_json3)
# Print the JSON output
print(output_json4)
print(output_json5)

{
    "Shop Name": "MARCO POLO ORTIGAS MANILA",
    "Owner Name": "FRONTIER ORTIGAS",
    "Address": {
        "Organization": "FRONTIER ORTIGAS",
        "Street": "HOTEL and RESORT CORP.",
        "City": "Meralco Ave. & Sapphire Rd.",
        "Postal Code": "1400",
        "Country": "Philippines"
    },
    "Registration Number": "008-611-772-000"
}
{
    "Table": {
        "Name": "Bi",
        "Number": "333099"
    },
    "Staff": {
        "Name": "ALEMIR",
        "Cover": 3
    }
}
{
    "Date": "08 Feb,i5",
    "Time": "12:49"
}
{
    "Items": [
        {
            "Item Name": "0 JASMINE",
            "Price": 178.57
        },
        {
            "Item Name": "ST.CHX FEET x0",
            "Price": 160.71
        },
        {
            "Item Name": "ST.PORK SIOMAT",
            "Price": 196.43
        },
        {
            "Item Name": "ST.G RICE ABALONE",
            "Price": 196.43
        },
        {
            "Item Name": "ST.SPINACH DUMPLING \u00a9",
      

## Additional Notes
 here I tried with  other preprocessing techniques with de noising, border removal, Intensity transformations.      But those give the outputs which  are less perfect compare to this approach

 I append those codes as well.

In [ ]:
#approach one
import numpy as np

def mapp(h):
    h = h.reshape((4,2))
    hnew = np.zeros((4,2),dtype = np.float32)

    add = h.sum(1)
    hnew[0] = h[np.argmin(add)]
    hnew[2] = h[np.argmax(add)]

    diff = np.diff(h,axis = 1)
    hnew[1] = h[np.argmin(diff)]
    hnew[3] = h[np.argmax(diff)]

    return hnew



import cv2
import numpy as np
import matplotlib.pyplot as plt
image=cv2.imread("top_half.jpg")   #read in the image
image=cv2.resize(image,(700,800)) #resizing because opencv does not work well with bigger images
orig=image.copy()

gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)  #RGB To Gray Scale
thresh, im_bw = cv2.threshold(gray, 140, 220, cv2.THRESH_BINARY)
cv2.imwrite("bw_image.jpg", im_bw)

blurred=cv2.GaussianBlur(gray,(3,3),0.2)  #(5,5) is the kernel size and 0 is sigma that determines the amount of blur
#cv2.imshow("Blur",blurred)

edged=cv2.Canny(blurred,30,60)  #30 MinThreshold and 50 is the MaxThreshold
#cv2.imshow("Canny",edged)


contours,hierarchy=cv2.findContours(edged,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)  #retrieve the contours as a list, with simple apprximation model
contours=sorted(contours,key=cv2.contourArea,reverse=True)

#the loop extracts the boundary contours of the page
for c in contours:
    p=cv2.arcLength(c,True)
    approx=cv2.approxPolyDP(c,0.02*p,True)

    if len(approx)==4:
        target=approx
        break
approx=mapp(target) #find endpoints of the sheet

pts=np.float32([[0,0],[800,0],[800,800],[0,800]])  #map to 800*800 target window

op=cv2.getPerspectiveTransform(approx,pts)  #get the top or bird eye view effect
dst=cv2.warpPerspective(orig,op,(800,800))

plt.imshow(dst)
#cv2.imshow("Scanned",dst)
# press q or Esc to close
#cv2.waitKey(0)
#cv2.destroyAllWindows()




#approach two

#transform = np.arange(255,-1,-1).astype("uint8")
#gray = cv2.LUT(gray, transform)
#cv2.imshow("Title",gray)
def noise_removal(image):
    import numpy as np
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.dilate(image, kernel, iterations=1)
    kernel = np.ones((1, 1), np.uint8)
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    image = cv2.medianBlur(image, 3)
    return (image)

def remove_borders(image):
    contours, heiarchy = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cntsSorted = sorted(contours, key=lambda x:cv2.contourArea(x))
    cnt = cntsSorted[-1]
    x, y, w, h = cv2.boundingRect(cnt)
    crop = image[y:y+h, x:x+w]
    return (crop)

no_noise = noise_removal(im_bw)
cv2.imwrite("no_noise.jpg", no_noise)
no_borders = remove_borders(im_bw)
cv2.imwrite("no_borders.jpg", no_borders)
display('no_borders.jpg')
plt.imshow(no_borders)

